In [ ]:
import torch
import torchvision
from torch.nn import functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
import cv2
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
import os

os.environ['HTTP_PROXY'] = 'http://fp.cs.ovgu.de:3210/'
os.environ['HTTPS_PROXY'] = 'http://fp.cs.ovgu.de:3210/'

# !git clone https://github.com/yiskw713/ScoreCAM.git

In [ ]:
%cd /project/validating_attribution_techniques/commons/ScoreCAM

In [ ]:
!nvidia-smi

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"
ngpu = torch.cuda.device_count()
for gpu_id in range(ngpu):
    gpu = torch.cuda.get_device_name(gpu_id)
    print(f"GPU:{gpu_id} {gpu}")

In [ ]:
%matplotlib inline

import numpy as np
import skimage.transform
import torch
import torch.nn as nn
import torch.nn.functional as F
# from ScoreCAM import cam

from PIL import Image
from matplotlib.pyplot import imshow
from torchvision import models, transforms
from torchvision.utils import save_image

from cam import CAM, GradCAM, GradCAMpp, SmoothGradCAMpp, ScoreCAM
from utils.visualize import visualize, reverse_normalize
from utils.imagenet_labels import label2idx, idx2label

# Loading Original Image

In [ ]:
image = Image.open('/project/validating_attribution_techniques/saikat/Source_images/Original_image/cat_dog.jpeg')
imshow(image)

# Preprocessing Test Image

In [ ]:
def preprocessing(image):
    # Resize the image
    size = (224, 224)
    image = image.resize(size)
    
    # preprocessing. mean and std from ImageNet
    normalize = transforms.Normalize(
       mean=[0.485, 0.456, 0.406],
       std=[0.229, 0.224, 0.225]
    )

    preprocess = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

    # convert image to tensor
    tensor = preprocess(image)

    # reshape 4D tensor (N, C, H, W)
    tensor = tensor.unsqueeze(0)
    
    return tensor

In [ ]:
tensor = preprocessing(image)
tensor.shape

In [ ]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True).to(device)

In [ ]:
gpu_reference_tensor = next(model.parameters())
tensor = tensor.type_as(gpu_reference_tensor)

# Model Target Layer

In [ ]:
target_layer = model.layer4[1].conv2

In [ ]:
classes = open("/project/validating_attribution_techniques/shardul/classes.txt").read().splitlines()

# Attribution Technique Function: GradCAM

In [ ]:
def gradCAM(target_layer, model, tensor):
    wrapped_model = GradCAM(model, target_layer)
    cam, idx = wrapped_model(tensor)
    return cam, idx

# Attribution Technique Function: ScoreCAM

In [ ]:
def scoreCAM(target_layer, model, tensor):
    wrapped_model = ScoreCAM(model, target_layer)
    cam, idx = wrapped_model(tensor)
    return cam, idx

# Attribution Technique Function: SmoothGradCAMpp

In [ ]:
def smoothGradCAMpp(target_layer, model, tensor):
    wrapped_model = SmoothGradCAMpp(model, target_layer, n_samples=25, stdev_spread=0.15)
    cam, idx = wrapped_model(tensor)
    return cam, idx

# Saliency Map Generation Function

In [ ]:
def saliency_map(tensor, cam):
    tensor = tensor.cpu()
    img = reverse_normalize(tensor)
    heatmap = visualize(img, cam)
    heatmap = np.transpose(heatmap.squeeze(), (1, 2, 0))
    return heatmap

# GradCAM on Original Image

In [ ]:
cam, idx = gradCAM(target_layer, model, tensor)
cam = cam.cpu()
print(idx)
print(idx2label[idx])

In [ ]:
fig, ax = plt.subplots()
heatmap = saliency_map(tensor, cam)
ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
ax.set_title(f"Class: {classes[idx]}")
fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/GradCAM/Original_image/{idx2label[idx]}.jpeg")

# ScoreCAM on Original Image

In [ ]:
cam, idx = scoreCAM(target_layer, model, tensor)
cam = cam.cpu()
print(idx)
print(idx2label[idx])

In [ ]:
fig, ax = plt.subplots()
heatmap = saliency_map(tensor, cam)
ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
ax.set_title(f"Class: {classes[idx]}")
fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/ScoreCAM/Original_image/{idx2label[idx]}.jpeg")

# SmoothGradCAM on Original Image

In [ ]:
cam, idx = smoothGradCAMpp(target_layer, model, tensor)
cam = cam.cpu()
print(idx)
print(idx2label[idx])

In [ ]:
fig, ax = plt.subplots()
heatmap = saliency_map(tensor, cam)
ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
ax.set_title(f"Class: {classes[idx]}")
fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/SmoothGradCAMpp/Original_image/{idx2label[idx]}.jpeg")

# Creating and Saving Occluded Images

In [ ]:
#Creating the patch
PATCH_SIZE = 60
def apply_grey_patch(path, image, top_left_x, top_left_y, patch_size):
    patched_image = np.array(image, copy=True)
    patched_image[top_left_y:top_left_y + patch_size, top_left_x:top_left_x + patch_size, :] = 0
    img = keras.preprocessing.image.array_to_img(patched_image)
    print(path)
    img.save(path)
    print(np.shape(patched_image))
    return patched_image

In [ ]:
#Putting the patch over the image
image_path = "/project/validating_attribution_techniques/saikat/Source_images/Original_image/cat_dog.jpeg"
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image = tf.keras.preprocessing.image.img_to_array(image)

i = 0
# Iterate the patch over the image
for top_left_x in range(0, image.shape[0], PATCH_SIZE):
    for top_left_y in range(0, image.shape[1], PATCH_SIZE):
        # Apply the patch and display the image
        path = "/home/smitra/project/validating_attribution_techniques/saikat/Source_images/Occluded_images/occluded_img_" + str(i) + ".jpg"
        i+=1
        patched_image = apply_grey_patch(path,image, top_left_x, top_left_y, PATCH_SIZE)
        patched_image = patched_image.astype('float32') / 255.0
#         n_patched_image 

# Finding out number of Occluded Images generated

In [ ]:
dir_path = "/home/smitra/project/validating_attribution_techniques/saikat/Source_images/Occluded_images/"
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

# GradCAM on Occluded Images

In [ ]:
fig, ax = plt.subplots()

for i in range(0,count):
    img = tf.keras.preprocessing.image.load_img("/home/smitra/project/validating_attribution_techniques/saikat/Source_images/Occluded_images/occluded_img_" + str(i) + ".jpg")
    tensor = preprocessing(img) # calling the preprocessing function
    gpu_reference_tensor = next(model.parameters())
    tensor = tensor.type_as(gpu_reference_tensor)
    cam, idx = gradCAM(target_layer, model, tensor)
    cam = cam.cpu()
    print(idx)
    print(idx2label[idx])
    heatmap = saliency_map(tensor, cam)
    ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
    ax.set_title(f"Class: {idx2label[idx]}")
    fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/GradCAM/Occluded_images/{i}_{idx2label[idx]}.jpg")

# ScoreCAM on Occluded Images

In [ ]:
fig, ax = plt.subplots()

for i in range(0,count):
    img = tf.keras.preprocessing.image.load_img("/home/smitra/project/validating_attribution_techniques/saikat/Source_images/Occluded_images/occluded_img_" + str(i) + ".jpg")
    tensor = preprocessing(img) # calling the preprocessing function
    gpu_reference_tensor = next(model.parameters())
    tensor = tensor.type_as(gpu_reference_tensor)
    cam, idx = scoreCAM(target_layer, model, tensor)
    cam = cam.cpu()
    print(idx)
    print(idx2label[idx])
    heatmap = saliency_map(tensor, cam)
    ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
    ax.set_title(f"Class: {idx2label[idx]}")
    fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/ScoreCAM/Occluded_images/{i}_{idx2label[idx]}.jpg")

# SmoothGradCAM on Occluded Images

In [ ]:
fig, ax = plt.subplots()

for i in range(0,count):
    img = tf.keras.preprocessing.image.load_img("/home/smitra/project/validating_attribution_techniques/saikat/Source_images/Occluded_images/occluded_img_" + str(i) + ".jpg")
    tensor = preprocessing(img) # calling the preprocessing function
    gpu_reference_tensor = next(model.parameters())
    tensor = tensor.type_as(gpu_reference_tensor)
    cam, idx = smoothGradCAMpp(target_layer, model, tensor)
    cam = cam.cpu()
    print(idx)
    print(idx2label[idx])
    heatmap = saliency_map(tensor, cam)
    ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
    ax.set_title(f"Class: {idx2label[idx]}")
    fig.savefig(f"/home/smitra/project/validating_attribution_techniques/saikat/Results/SmoothGradCAMpp/Occluded_images/{i}_{idx2label[idx]}.jpg")

# Adding Guassian Noise

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

mean = 0
variance = 1.5
stddev = variance ** 0.5
noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)
noise = np.uint8(noise)  # Convert noise array to uint8

# Add noise to the image using element-wise addition
image_with_noise = np.clip(image + noise, 0, 255).astype(np.uint8)

imshow(image_with_noise)

In [ ]:
# Resize the image
size = (224, 224)
image = image.resize(size)
    
# preprocessing. mean and std from ImageNet
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

preprocess = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

# convert image to tensor
tensor = preprocess(image)

# reshape 4D tensor (N, C, H, W)
tensor = tensor.unsqueeze(0)
    
return tensor

gpu_reference_tensor = next(model.parameters())
tensor = tensor.type_as(gpu_reference_tensor)

# GradCAM on Noisy Image

In [ ]:
cam, idx = smoothGradCAMpp(target_layer, model, tensor)
cam = cam.cpu()
print(idx)
print(idx2label[idx])

In [ ]:
fig, ax = plt.subplots()
heatmap = saliency_map(tensor, cam)
ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
ax.set_title(f"Class: {classes[idx]}")
fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/GradCAM/Noisy_image/{idx2label[idx]}.jpeg")

# ScoreCAM on Noisy Image

In [ ]:
cam, idx = smoothGradCAMpp(target_layer, model, tensor)
cam = cam.cpu()
print(idx)
print(idx2label[idx])

In [ ]:
fig, ax = plt.subplots()
heatmap = saliency_map(tensor, cam)
ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
ax.set_title(f"Class: {classes[idx]}")
fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/ScoreCAM/Noisy_image/{idx2label[idx]}.jpeg")

# SmoothGradCAMpp on Noisy Image

In [ ]:
cam, idx = smoothGradCAMpp(target_layer, model, tensor)
cam = cam.cpu()
print(idx)
print(idx2label[idx])

In [ ]:
fig, ax = plt.subplots()
heatmap = saliency_map(tensor, cam)
ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
ax.set_title(f"Class: {classes[idx]}")
fig.savefig(f"/project/validating_attribution_techniques/saikat/Results/SmoothGradCAMpp/Noisy_image/{idx2label[idx]}.jpeg")

# KL Divergence between Original and Occluded Saliency

# Pearson Correlation Coefficient between Original and Occluded Saliency

# KL Divergence between Original and Noisy Saliency

# Pearson Correlation Coefficient between Original and Noisy Saliency